<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/Standard_banner.png' align='right' width='100%'/>

<a href="../Index.ipynb"><< Index</a>
<br>
<a href="./1_2_SRAL_file_structure.ipynb">Understanding SRAL product structure >></a>

<font color="#138D75">**Copernicus Marine Training Service**</font> <br>
**Copyright:** 2024 EUMETSAT <br>
**License:** MIT <br>
**Authors:** Ben Loveday (EUMETSAT/Innoflair UG), Hayley Evers-King (EUMETSAT), Vinca Rosmorduc (CLS)

<html>
  <div style="width:100%">
    <div style="float:left"><a href="https://mybinder.org/v2/git/https%3A%2F%2Fgitlab.eumetsat.int%2Feumetlab%2Foceans%2Focean-training%2Fsensors%2Flearn-sral/HEAD?urlpath=%2Ftree%2F1_SRAL_introductory%2F1_1b_SRAL_data_access_HDA.ipynb"><img src="https://mybinder.org/badge_logo.svg" alt="Open in Binder"></a></div>
    <div style="float:left"><p>&emsp;</p></div>
  </div>
  <div style="width:100%">
    <div style="float:left"><a href="https://jupyterhub-wekeo.apps.eumetsat.dpi.wekeo.eu/hub/user-redirect/lab/tree/public/wekeo4oceans/learn-sral/1_SRAL_introductory/1_1b_SRAL_data_access_HDA.ipynb"><img src="https://img.shields.io/badge/launch-WEKEO-1a4696.svg?style=flat&logo=" alt="Open in WEkEO"></a></div>
    <div style="float:left"><p>&emsp;</p></div>
  </div>    
</html>

<div class="alert alert-block alert-success">
<h3>Learn SRAL: Introductory</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
This notebook has the following prerequisites:
- **<a href="https://my.wekeo.eu/user-registration" target="_blank">A WEkEO account</a>** to use WEkEO.  

There are no prerequisite notebooks for this module.
</div>
<hr>

# 1.1b Accessing SRAL data via the WEkEO HDA adaptor

### Data used

| Product Description  | WEkEO HDA ID | WEkEO metadata |
|:--------------------:|:-----------------------:|:-------------:|
| Sentinel-3 SRAL level-1A | EO:EUM:DAT:SENTINEL-3:SR_1_SRA_A_ |<a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AEUM%3ADAT%3ASENTINEL-3%3ASR_1_SRA_A_" target="_blank">link</a>|
| Sentinel-3 SRAL level-1BS | EO:EUM:DAT:SENTINEL-3:SR_1_SRA_BS | <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AEUM%3ADAT%3ASENTINEL-3%3ASR_1_SRA_BS" target="_blank">link</a>|
| Sentinel-3 SRAL level-1B | EO:EUM:DAT:SENTINEL-3:SR_1_SRA___ | <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AEUM%3ADAT%3ASENTINEL-3%3ASR_1_SRA___" target="_blank">link</a> |
| Sentinel-3 SRAL level-2 | EO:EUM:DAT:SENTINEL-3:SR_2_WAT___ | <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AEUM%3ADAT%3ASENTINEL-3%3ASR_2_WAT___" target="_blank">link</a>|
| Global ocean gridded L4 sea surface heights and derived variables NRT| EO:MO:DAT: SEALEVEL_GLO_PHY_L4_NRT_OBSERVATIONS_008_046 | <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AMO%3ADAT%3ASEALEVEL_GLO_PHY_L4_NRT_OBSERVATIONS_008_046" target="_blank">link</a>|


### Learning outcomes

At the end of this notebook you will know;
* how to download data from WEkEO using the <font color="#138D75">**harmonised data access (hda)**</font> client

### Outline

Data from SRAL is available through multiple sources, either via a web user interface (WebUI) or through code and command line interfaces with an Application Programming Interface (API). WebUIs are useful for accessing quick-look data visualisations, and for browsing to see what is available. APIs are more useful for routine, automated and operational data access. Here we will guide you through ways you can access SRAL data through both methods.

Users should note that this notebook will use WEkEO [harmonised data access (HDA) API](#section3) to download <font color="#138D75">**all**</font> of the products required for the lessons in the SRAL introductory module. The HDA adaptor is the fastest and most suitable way of retrieving SRAL data when working inside the WEkEO infrastructure, either on the JupyterHub or on virtual machines.

<div class="alert alert-info" role="alert">

## <a id='TOC-TOP'></a>Contents

</div>
    
 1. [Download data via GUIs](#section1)
 1. [The products we need](#section2)
 1. [Downloading via the WEkEO API](#section3)

<hr>

We begin by importing all of the libraries that we need to run this notebook. If you have built your python using the environment file provided in this repository, then you should have everything you need. For more information on building environment, please see the repository **<a href="../README.md" target="_blank">README</a>**.

In [ ]:
import configparser # a library that allows us to parse standard configuration files
import IPython      # a library that helps us display video and HTML content
import os           # a library that allows us access to basic operating system commands like making directories
import json         # a library that helps us make JSON format files
import shutil       # a library that allows us access to basic operating system commands like copy
import zipfile      # a library that allows us to unzip zip-files.
import hda          # a library for downloading via wekeo
import glob         # a library that helps us find files

Sometimes we use configuration files to help us set some notebook parameters. The box below reads a configuration file to help us decide how large to make the videos displayed below.

In [ ]:
config = configparser.ConfigParser()
config["nbook"] = {"v_wd" : "700", "v_ht" : "450"}
if os.path.exists(os.path.join(os.path.dirname(os.getcwd()),"frameworks","config.ini")):
    config.read(os.path.join(os.path.dirname(os.getcwd()),"frameworks","config.ini"))

Next we will create a download directory to store the products we will download in this notebook.

In [ ]:
# Create a download directory for our SRAL products
download_dir = os.path.join(os.getcwd(), "products")
os.makedirs(download_dir, exist_ok=True)

<div class="alert alert-info" role="alert">

## <a id='section1'></a>1. Downloading data via GUIs
[Back to top](#TOC-TOP)

</div>

<hr>

### WEkEO

WEkEO is the Copernicus Data Information and Access Service (DIAS) reference service for environmental data. It is a partnership between EUMETSAT, the European Centre for Medium-range Weather Forecasting (ECMWF), Mercator Ocean International, and the European Environment Agency (EEA). You can see an overview of the WEkEO service offering in the video below. WEkEO offers a harmonised data access (HDA) solution to a wide range of Copernicus data, including that from Sentinel-3 and associated down-stream products from the Copernicus Marine Service. The WEkEO HDA can be accessed via the <a href="https://www.wekeo.eu/data" target="_blank">WEkEO Data Viewer</a> and through an API, see [Downloading from WEkEO via the API](#section4).

You can visit the <a href="https://www.wekeo.eu/data" target="_blank">WEkEO Data Viewer</a> and explore the available marine data, however this notebook will also show you how to use the HDA API - see [Downloading from WEkEO via the API](#section4) below.

#### Links:
* <a href="https://www.wekeo.eu/data" target="_blank">WEkEO Data Viewer</a>
* <a href="https://www.wekeo.eu/docs" target="_blank">More information on WEkEO</a>

In [ ]:
IPython.display.YouTubeVideo('UOexmq1LdyY', width=config["nbook"]["v_wd"], height=config["nbook"]["v_ht"])

<div class="alert alert-info" role="alert">

## 2. <a id='section2'></a>The products we need for this module
[Back to top](#TOC-TOP)
    
</div>

We need a number of SRAL level-1 and level-2 products for the notebooks within the introductory part of the learn-sral module. These products are shown below, and we will retrieve these products from the WEkEO product catalogue.
* S3B_SR_1_SRA____20240206T104924_20240206T113954_20240303T021922_3029_089_208______MAR_O_NT_005.SEN3
* S3B_SR_2_WAT____20240206T104925_20240206T113638_20240303T024359_2833_089_208______MAR_O_NT_005.SEN3
* S3A_SR_2_WAT____20240202T062911_20240202T071431_20240227T220310_2720_108_291______MAR_O_NT_005.SEN3
* S3A_SR_2_WAT____20240201T193847_20240201T202315_20240227T113018_2668_108_284______MAR_O_NT_005.SEN3  

<hr>
<div class="alert alert-info" role="alert">

## 3. <a id='section3'></a>Downloading from WEkEO via the API
[Back to top](#TOC-TOP)

</div>


Downloading from WEkEO harmonised data access (HDA) API is facilitated by a data access client. 
If you are currently working on the WEkEO JupyterHub, this client will already be installed. Otherwise, if you are working with the recommended Anaconda Python distribution and used the supplied environment file (environment.yml) to build this python environment (as detailed in the README), you will already have installed this. If not, you can find the source code <a href="https://github.com/ecmwf/hda" target="_blank">here</a>.

In order to allow us to download data using the WEkEO HDA API, we need to provide our credentials. We can do this in two ways;
* **Option 1** - by creating a configuration file (*recommended, and user here with the default configuration file*)
* **Option 2** - by supplying our credentials directly in this script (*not recommended, but sometimes useful*)

#### Option 1: creating a credentials file.

By default, the HDA API expects the configuration to be called `.hdarc`, and to reside in our home directory. For most computer systems the home directory can be found at the path \user\username, /users/username, or /home/username depending on your operating system. In this file we need to add the following information exactly as follows;

```
url: https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker
user: <your_user_name>
password: <your_password>
```

You must replace `<your_user_name>` and `<your_password>` with the information from your WEkEO account (if you don't have one yet, register <a href="https://www.wekeo.eu/" target="_blank">here</a>. Once you have entered these credentials in the file, the `hda` client will automatically read in the credentials from the file when you use it. Make sure to save the file without any kind of extension.

If you are using the default approach you can establish an instance of the client like so;

`c = hda.Client()`

Alternatively, if you wish to specify your own configuration file, you can do so by adapting the code line below. The format should be the same as specified above.

`c = hda.Client(hda.Configuration(path=<your_config_file>)`

Where you should replace `<your_config_file>` with the path to your configuration file, for example;
* "myconfig.txt" if it is in this directory.
* "/users/username/myconfig.txt" as an example of an absolute path to a file on Linux and/or OSx.
* os.path.join("users","username","myconfig.txt") as an example of an absolute path on any platform.
* os.path.join(os.path.expanduser("~"), "myconfig.txt")) if it is in your home directory, on all operating systems.

#### Option 2: provide credentials directly

You can provide your credentials directly as follows;

`c = hda.Client(hda.Configuration(url="https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker",
                                 user="<your_user_name>",
                                 password="<your_password>"))`
                                 
*Note: this method is convenient in the short term, but is not really recommended as you have to put your user name and password in this notebook, and run the risk of accidentally sharing them. This method also requires you to authenticate on a notebook-by-notebook basis.*

In [ ]:
c = hda.Client()

The WEkEO HDA client accepts requests as JSON queries. These have a specific format, which may look complex, but you can build on the examples you can find in the GUI, available under the **Show API request** button;

<img src='../img/WEkEO_show_api_SRAL.png' align='centre' width='75%'></img>

<center>Note: the WEkEO viewer is unable to display along-track SRAL products*</center>

WEkEO allows you to search for data in time and space, but not by product name. However, we can work around this by providing the exact time stamps for our data. Below, we have a query for the first item in our product list, the SRAL level 1B product. You can see that the start and end times match the filename times, and the `producttype`, and `timeliness` correspond to those in the earlier specified filename too.

In [ ]:
query = {
  "datasetId": "EO:EUM:DAT:SENTINEL-3:SR_1_SRA___",
  "dateRangeSelectValues": [
    {
      "name": "position",
      "start": "2022-02-18T11:03:59.000Z",
      "end": "2022-02-18T11:04:00.000Z"
    }
  ],
  "stringChoiceValues": [{"name": "producttype", "value": "SR_1_SRA___"},
                         {"name": "timeliness", "value": "NT"},
                         {"name": "relorbit", "value": "365"}
  ]
}

This next section of code prints the names of the files that have been found during the search...

In [ ]:
matches = c.search(query)
for match in matches.results:
    fdst = match['filename']
    print(f"Found: {fdst}")

We can see that we have found the file we are interested in, so the next section of code can be used to download it...

In [ ]:
matches.download()

Finally, we can unzip the file.

In [ ]:
with zipfile.ZipFile(fdst, 'r') as zip_ref:
    zip_ref.extractall(download_dir)
    print(f'Unzipping of product {fdst} finished.')
os.remove(fdst)

Now lets download our other three products. We will need to set up query files for each.

In [ ]:
query_WAT1 = {
  "datasetId": "EO:EUM:DAT:SENTINEL-3:SR_2_WAT___",
  "dateRangeSelectValues": [{"name": "position", "start": "2022-02-18T11:04:00.000Z", "end": "2022-02-18T11:04:01.000Z"}],
  "stringChoiceValues": [{"name": "producttype", "value": "SR_2_WAT___"}, {"name": "timeliness", "value": "NT"}, {"name": "relorbit", "value": "365"}]
}

query_WAT2 = {
  "datasetId": "EO:EUM:DAT:SENTINEL-3:SR_2_WAT___",
  "dateRangeSelectValues": [{"name": "position", "start": "2022-01-03T06:32:57.000Z", "end": "2022-01-03T06:32:58.000Z"}],
  "stringChoiceValues": [{"name": "producttype", "value": "SR_2_WAT___"}, {"name": "timeliness", "value": "NT"}, {"name": "relorbit", "value": "234"}]
}

query_WAT3 = {
  "datasetId": "EO:EUM:DAT:SENTINEL-3:SR_2_WAT___",
  "dateRangeSelectValues": [{"name": "position", "start": "2022-01-01T20:08:35.000Z", "end": "2022-01-01T20:08:36.000Z"}],
  "stringChoiceValues": [{"name": "producttype", "value": "SR_2_WAT___"}, {"name": "timeliness", "value": "NT"}, {"name": "relorbit", "value": "213"}]
}

queries = [query_WAT1, query_WAT2, query_WAT3]

And again we can search for, and download the results...

In [ ]:
for query in queries:
    matches = c.search(query)
    matches.download()
    for match in matches.results:
        fdst = match['filename']
        print(f"Found: {fdst}")        
        with zipfile.ZipFile(fdst, 'r') as zip_ref:
            zip_ref.extractall(download_dir)
            print(f'Unzipping of product {fdst} finished.')
        os.remove(fdst)

Lastly, we will get some background data for these exercises from the Copernicus Marine Service catalogue, through WEkEO. This is a level-3 product, which is distributed on a consistent grid. The query below is slightly different in its construction to what we used above, but the principle if the same. If we wished, we could gather a "cube" of data over an extended time period, but in this case we will request a single 7-day product.

In [ ]:
query_GEO = {
  "datasetId": "EO:MO:DAT:SEALEVEL_GLO_PHY_L4_NRT_OBSERVATIONS_008_046:dataset-duacs-nrt-global-merged-allsat-phy-l4_202112",
  "boundingBoxValues": [{"name": "bbox", "bbox": [20, -52, 50, 0]}],
  "dateRangeSelectValues": [{"name": "time", "start": "2022-01-02T00:00:00.000Z", "end": "2022-01-02T00:00:00.000Z"}],
  "multiStringSelectValues": [{"name": "variable", "value": ["ugosa", "vgosa", "sla", "longitude", "latitude"]}]
}

matches = c.search(query_GEO)
matches.download()
fdst = glob.glob(matches.results[0]['productInfo']['product'].split(':')[0] + '*')
os.rename(fdst[0], os.path.join(download_dir, fdst[0].split('?')[0]))

You can now continue to work through the rest of the notebooks in this repository and learn more about working with SRAL data.

<hr>
<a href="../Index.ipynb"><< Index</a>
<br>
<a href="./1_2_SRAL_file_structure.ipynb">Understanding SRAL product structure >></a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean" target="_blank">View on GitLab</a> | <a href="https://training.eumetsat.int/" target="_blank">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int target="_blank">Contact helpdesk for support </a> | <a href=mailto:training@eumetsat.int target="_blank">Contact our training team to collaborate on and reuse this material</a></span></p>